In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
# import pyspark.sql as ps

In [2]:
spark = SparkSession.builder.master('local').appName("hw33").config('spark.driver.bindAddress', 'localhost').config('spark.ui.port', '4040').getOrCreate()

22/12/06 11:27:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


 Создайте схему будущего фрейма данных. Схема должна включать следующие атрибуты:

·   id -  уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.

·   timestamp – дата и время события в формате unix timestamp.

·   type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).

·   page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.

·   tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.

·   sign – наличие у пользователя личного кабинета. Значения – True/False.

### Создание схемы DF

In [3]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('timestamp', StringType(), True),
    StructField('type', StringType(), True), 
    StructField('page_id', IntegerType(), True),
    StructField('tag', StringType(), True),
    StructField('sign', BooleanType(), True),
])

Создайте датафрейм с описанной выше схемой данных.

Наполните датафрейм данными. Пример:

(12345, 1667627426, "click", 101, "Sport”, False)

### Наполнение DF

In [4]:
data = [
    (45445, 1670198400, "visit", 565, 'Sport', False),
    (45445, 1670198600, "click", 565, 'Sport', False),
    (45445, 1670191400, "move", 100, 'IT', False),
    (45445, 1670128400, "scroll", 99, 'IT', False),
    (45445, 1670298400, "visit", 2, 'Music', False),
    
    (12587, 1670140800, "visit", 178, 'Art', True),
    (12587, 1670140804, "click", 178, 'Art', True),
    (12587, 1670140812, "click", 178, 'Art', True),
    (12587, 1670140813, "scroll", 178, 'Art', True),
    (12587, 1670140815, "scroll", 178, 'Art', True),
    
    (68851, 1669893060, "visit", 222, 'Music', False),
    (32856, 1659893060, "visit", 565, 'Sport', False),
    (9547, 1659893060, "visit", 331, 'Politics', False),
    
    (58741, 1669633860, "visit", 565, 'Sport', True),
    (58741, 1669633861, "move", 222, 'Music', True),
    
    (74859, 1670077320, "visit", 100, 'IT', False),
    (74859, 1670077360, "click", 100, 'IT', False),
    (74859, 1670077361, "move", 565, 'Sport', False),
    (74859, 1670077373, "scroll", 565, 'Sport', False),
    (74859, 1670077790, "move", 331, 'Politics', False),
]

### Создание, отображение DF

In [5]:
df = spark.createDataFrame(data=data,schema=schema)

In [6]:
df.show()

+-----+----------+------+-------+--------+-----+
|   id| timestamp|  type|page_id|     tag| sign|
+-----+----------+------+-------+--------+-----+
|45445|1670198400| visit|    565|   Sport|false|
|45445|1670198600| click|    565|   Sport|false|
|45445|1670191400|  move|    100|      IT|false|
|45445|1670128400|scroll|     99|      IT|false|
|45445|1670298400| visit|      2|   Music|false|
|12587|1670140800| visit|    178|     Art| true|
|12587|1670140804| click|    178|     Art| true|
|12587|1670140812| click|    178|     Art| true|
|12587|1670140813|scroll|    178|     Art| true|
|12587|1670140815|scroll|    178|     Art| true|
|68851|1669893060| visit|    222|   Music|false|
|32856|1659893060| visit|    565|   Sport|false|
| 9547|1659893060| visit|    331|Politics|false|
|58741|1669633860| visit|    565|   Sport| true|
|58741|1669633861|  move|    222|   Music| true|
|74859|1670077320| visit|    100|      IT|false|
|74859|1670077360| click|    100|      IT|false|
|74859|1670077361|  

### Отображение схемы DF

In [7]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- sign: boolean (nullable = true)



### Приведение к timestamp 

In [8]:
df = df.select(df.columns).withColumnRenamed('timestamp', 'unix_time')

df = df.select(* df.columns).withColumn('timestamp',from_unixtime(col("unix_time"),"yyyy-MM-dd HH:mm:ss"))

In [9]:
df.show(5)

+-----+----------+------+-------+-----+-----+-------------------+
|   id| unix_time|  type|page_id|  tag| sign|          timestamp|
+-----+----------+------+-------+-----+-----+-------------------+
|45445|1670198400| visit|    565|Sport|false|2022-12-05 00:00:00|
|45445|1670198600| click|    565|Sport|false|2022-12-05 00:03:20|
|45445|1670191400|  move|    100|   IT|false|2022-12-04 22:03:20|
|45445|1670128400|scroll|     99|   IT|false|2022-12-04 04:33:20|
|45445|1670298400| visit|      2|Music|false|2022-12-06 03:46:40|
+-----+----------+------+-------+-----+-----+-------------------+
only showing top 5 rows



**Решите следующие задачи:**

·   Вывести топ-5 самых активных посетителей сайта

In [10]:
df.groupby("id").count().orderBy(desc("count")).show(5)

+-----+-----+
|   id|count|
+-----+-----+
|12587|    5|
|45445|    5|
|74859|    5|
|58741|    2|
|32856|    1|
+-----+-----+
only showing top 5 rows



·   Посчитать процент посетителей, у которых есть ЛК

In [11]:
num_with_acc = df.filter(df.sign == True).groupby("id").count().distinct().count()
share_with_acc = int(df.filter(df.sign == True).groupby("id").count().distinct().count() / df.groupby("id").count().distinct().count()*100)

f"There are {num_with_acc} users with personal acconts. It's {share_with_acc} % from total amount of users."

"There are 2 users with personal acconts. It's 28 % from total amount of users."

·   Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице

In [12]:
df.filter(df.type == "click").groupby("page_id").count().orderBy("count", ascending = False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    178|    2|
|    100|    1|
|    565|    1|
+-------+-----+



·   Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

In [13]:
df_with_time_range = df.withColumn("time_range", round(hour("timestamp")/4))
df_with_time_range.show(5)

+-----+----------+------+-------+-----+-----+-------------------+----------+
|   id| unix_time|  type|page_id|  tag| sign|          timestamp|time_range|
+-----+----------+------+-------+-----+-----+-------------------+----------+
|45445|1670198400| visit|    565|Sport|false|2022-12-05 00:00:00|       0.0|
|45445|1670198600| click|    565|Sport|false|2022-12-05 00:03:20|       0.0|
|45445|1670191400|  move|    100|   IT|false|2022-12-04 22:03:20|       6.0|
|45445|1670128400|scroll|     99|   IT|false|2022-12-04 04:33:20|       1.0|
|45445|1670298400| visit|      2|Music|false|2022-12-06 03:46:40|       1.0|
+-----+----------+------+-------+-----+-----+-------------------+----------+
only showing top 5 rows



·   Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

In [14]:
df_with_time_range.groupby("time_range").count().orderBy("count", ascending = False).show(1)

+----------+-----+
|time_range|count|
+----------+-----+
|       4.0|    7|
+----------+-----+
only showing top 1 row



·   Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов

1. Id – уникальный идентификатор личного кабинета

2. User_id – уникальный идентификатор посетителя

3. ФИО посетителя

4. Дату рождения посетителя 

5. Дата создания ЛК

### Создание схемы DF личных кабинетов пользователей

In [15]:
schema_users = StructType([
    StructField('id', IntegerType(), True),
    StructField('User_id', IntegerType(), True),
    StructField('name', StringType(), True), 
    StructField('dob', StringType(), True),
    StructField('date_create', StringType(), True),
])

### Наполнение DF личных кабинетов пользователей

In [16]:
data_users = [
    (778, 12587, "Grey Wolf", "881864520","1670077790"),
    (112, 58741, "White Rabbit", "661122060", "1575697560"),
    (11251, 866124, "Brown Bear", "251129640", "1638745140"),
    (13218, 89882, "Red Fox", "944921640", "1638745140")
]

### Создание DF личных кабинетов пользователей

In [17]:
df_users = spark.createDataFrame(data=data_users,schema=schema_users)

In [18]:
df_users.show()

+-----+-------+------------+---------+-----------+
|   id|User_id|        name|      dob|date_create|
+-----+-------+------------+---------+-----------+
|  778|  12587|   Grey Wolf|881864520| 1670077790|
|  112|  58741|White Rabbit|661122060| 1575697560|
|11251| 866124|  Brown Bear|251129640| 1638745140|
|13218|  89882|     Red Fox|944921640| 1638745140|
+-----+-------+------------+---------+-----------+



Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

In [19]:
#объединим два DF с личными кабинетами и статистикой действий и выведем новый DF
df_unit = df_users.select(df_users.columns)
df_unit =  df_unit.join(df_with_time_range, df_unit.User_id == df_with_time_range.id)
df_unit.show()

+---+-------+------------+---------+-----------+-----+----------+------+-------+-----+----+-------------------+----------+
| id|User_id|        name|      dob|date_create|   id| unix_time|  type|page_id|  tag|sign|          timestamp|time_range|
+---+-------+------------+---------+-----------+-----+----------+------+-------+-----+----+-------------------+----------+
|778|  12587|   Grey Wolf|881864520| 1670077790|12587|1670140800| visit|    178|  Art|true|2022-12-04 08:00:00|       2.0|
|778|  12587|   Grey Wolf|881864520| 1670077790|12587|1670140804| click|    178|  Art|true|2022-12-04 08:00:04|       2.0|
|778|  12587|   Grey Wolf|881864520| 1670077790|12587|1670140812| click|    178|  Art|true|2022-12-04 08:00:12|       2.0|
|778|  12587|   Grey Wolf|881864520| 1670077790|12587|1670140813|scroll|    178|  Art|true|2022-12-04 08:00:13|       2.0|
|778|  12587|   Grey Wolf|881864520| 1670077790|12587|1670140815|scroll|    178|  Art|true|2022-12-04 08:00:15|       2.0|
|112|  58741|Whi

In [20]:
df_unit.filter(df_unit.tag == "Sport").select("name").show()

+------------+
|        name|
+------------+
|White Rabbit|
+------------+

